In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
from skimage.filters import threshold_multiotsu
from skimage.restoration import denoise_tv_chambolle
from skimage import measure, io, img_as_ubyte, color
from skimage.filters import threshold_otsu
from scipy import ndimage 
from skimage.filters.rank import entropy
from skimage.morphology import disk
import glob
from skimage.color import label2rgb, rgb2gray
import os

In [4]:
try:
    import cv2 as cv
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install opencv-python
    import cv2 as cv

def main():
    #Files name / paths
    filepath_original = "C:\\JupyterLab\\PCA\\Aula7\\OneDrive_2024-05-07_Fungos_LNBr\\Solid_Medium\\06_L11_Solid_CPD\\01_SE_008.tif" #Define o nome do arquivo da imagem dos cristais original.
    filepath = 'C:\\JupyterLab\\PCA\\Aula7\\OneDrive_2024-05-07_Fungos_LNBr\\Solid_Medium\\06_L11_Solid_CPD\\01_SE_008.tif' #Define o nome do arquivo da imagem a ser analisada.
        
    basename = os.path.basename(filepath_original)
    file_name = os.path.splitext(basename)[0]
    
    # Carrega a imagem
    orig = cv.imread(cv.samples.findFile(filepath_original), cv.IMREAD_COLOR) #Salva na variável orig a imagem original, sem filtros, direto da linha.
    without_filter = orig.copy() #img original para se desenhar em cima
    src = cv.imread(cv.samples.findFile(filepath), cv.IMREAD_COLOR) #Salva a imagem que será manipulada na variável 
                                                                    #src, assim temos a original e a manipulada (após filtros prévios)
                                                                    #separadamente.
    prior = src.copy() #img dada com filtros prévios para se trabalhar
            
    #Morphological gradient 
    blur = cv.GaussianBlur(src, (7, 7), 2)
    h, w = prior.shape[:2]

    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))
    morph_gradient = cv.morphologyEx(blur, cv.MORPH_GRADIENT, kernel) #img com filtro gradiente morfológico puro
    mgrad = morph_gradient.copy() #img com filtro gradiente morfológico para se continuar processo
    
    #Gaussian Filter
    gray = cv.cvtColor(mgrad, cv.COLOR_BGR2GRAY) #Transforma a imagem em preto e branco.
        
    gray_c = gray.copy() 

    #gray_c = cv.medianBlur(gray_c, 9) 
        
    #Irregular Shape Detection Hough Transform 
    blue_channel = src[:,:,0]
    #background = (blue_channel <= 58)
    #fungus = (blue_channel > 54)
    ret2, threshold2 = cv2.threshold(blue_channel,54,80,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    im_floodfill = threshold2.copy()

    h2, w2 = threshold2.shape[:2]
    mask = np.zeros((h2+2, w2+2), np.uint8)

    flood_fill = cv.floodFill(im_floodfill, mask, (0,0), 200)

    im_floodfillinv = cv.bitwise_not(im_floodfill)

    im_out = threshold2 | im_floodfillinv
    #im_out = im_floodfillinv | threshold2

    threshold_blur = cv.GaussianBlur(im_out, (3,3), 9) #Aplica o filtro Gaussiano, neste caso com um desvio padrão de 9 pixels.

    img_inv = cv2.bitwise_not(threshold_blur)

    #threshold_blur = cv.medianBlur(threshold2, 9) #Aplica o filtro Gaussiano, neste caso com um desvio padrão de 9 pixels.

    ##closing = cv.morphologyEx(threshold_blur, cv.MORPH_CLOSE, kernel, iterations=4)

    #opening = cv.morphologyEx(threshold_blur, cv.MORPH_OPEN, kernel,iterations=4)

    #dilation = cv.dilate(opening, kernel, iterations=1)

    #closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel, iterations=4)

    ##opening = cv.morphologyEx(closing, cv.MORPH_OPEN, kernel,iterations=4)

    ##dilation = cv.dilate(opening, kernel, iterations=1)


    #closing = cv.morphologyEx(threshold_blur, cv.MORPH_CLOSE, kernel, iterations=4)

    opening = cv.morphologyEx(img_inv, cv.MORPH_OPEN, kernel,iterations=3)
    
    erosion = cv.erode(opening, kernel, iterations=1)

    #dilation = cv.dilate(opening, kernel, iterations=1)

    closing = cv.morphologyEx(erosion, cv.MORPH_CLOSE, kernel, iterations=2)

    #opening = cv.morphologyEx(closing, cv.MORPH_OPEN, kernel,iterations=4)

    #dilation = cv.dilate(opening, kernel, iterations=1)

    contours, hierarchy = cv.findContours(erosion, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    shapes_detected = cv.drawContours(without_filter, contours, -1, (255,0,0), 3)

    
    if os.path.isdir(f'./images/{file_name}'):
        print("Pasta já existente! Será sobrescrito!")
    else:
        os.makedirs(f'./images/{file_name}') # Cria uma pasta respectiva à imagem em análise
    
    #Centers
    print("Contours:", contours)
    #contours_df = pd.DataFrame(contours)
    #np.savetxt(f'/home/gcc/Documents/vitoria_yumi_220056/MNC_sscFext/PDE/mnc_code/{file_name}/{file_name}_contours.txt', contours_df.values, fmt='%d')
    
    #Images
    im_orig = cv.resize(orig, (640,512))
    cv.imshow("Original", im_orig) #Mostra a imagem original
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_original.png', orig)
    
    im_wf = cv.resize(without_filter, (640,512))
    cv.imshow("Without filter", im_wf) #Mostra a imagem original com os centros
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_without-filter.png', without_filter)
    
    #im_pde = cv.resize(src, (640,512))
    #cv.imshow('PDE', im_pde) #Mostra a imagem com o filtro baseado em EDP aplicado
    #cv.imwrite(f'C:\\JupyterLab\\PCA\\Aula9\\{file_name}_pde.png', src)
    
    im_mgrad = cv.resize(morph_gradient, (640,512))
    cv.imshow('Morphological gradient', im_mgrad) #Mostra a imagem com o filtro Gradiente Morfológico aplicado
    cv.imwrite(f'.\\images\\{file_name}\{file_name}_morphological-gradient.png', morph_gradient)
    
    im_gray = cv.resize(gray, (640,512))
    cv.imshow("Processed", im_gray) #Mostra a imagem com o filtro Gaussiano aplicado
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_gaussian.png', gray)

    im_blue = cv.resize(blue_channel, (640,512))
    cv.imshow("Blue channel", im_blue) #Mostra a imagem com o filtro Gaussiano aplicado
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_blue.png', blue_channel)

    im_threshold = cv.resize(threshold2, (640,512))
    cv.imshow("Threshold", im_threshold) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_threshold.png', threshold2)

    im_flood_fill = cv.resize(im_floodfill, (640,512))
    cv.imshow("Floodfilled", im_flood_fill) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_floodfill.png', im_floodfill)


    im_flood_fill_inv = cv.resize(im_floodfillinv, (640,512))
    cv.imshow("Inverted Floodfilled", im_flood_fill_inv) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_floodfill_inv.png', im_floodfillinv)


    im_foreground = cv.resize(im_out, (640,512))
    cv.imshow("Foreground", im_foreground) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_foreground.png', im_out)


    im_threshold_blur = cv.resize(threshold_blur, (640,512))
    cv.imshow("Threshold + Blur", im_threshold_blur) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_threshold_blur.png', threshold_blur)

    im_inv = cv.resize(img_inv, (640,512))
    cv.imshow("Inverted", im_inv) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_threshold_blur.png', img_inv)


    im_opening = cv.resize(opening, (640,512))
    cv.imshow("Opening", im_opening) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_threshold_blur.png', opening)


    ##im_dilation = cv.resize(dilation, (640,512))
    ##cv.imshow("Dilation", im_dilation) #Mostra a imagem fornecida com filtros dilation.
    ##cv.imwrite(f'C:\\JupyterLab\\PCA\\Aula9\\{file_name}_dilation.png', dilation)

    im_erosion = cv.resize(erosion, (640,512))
    cv.imshow("Erosion", im_erosion) #Mostra a imagem fornecida com filtros THreshold.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_erosion.png', erosion)


    im_closing = cv.resize(closing, (640,512))
    cv.imshow("Closing", im_closing) #Mostra a imagem fornecida com filtros closing.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_threshold_blur.png', closing)


    im_shapes = cv.resize(shapes_detected, (640,512))
    cv.imshow("detected shapes", im_shapes) #Mostra a imagem fornecida com filtros prévios com os círculos detectados.
    cv.imwrite(f'.\\images\\{file_name}\\{file_name}_detected-shapes.png', shapes_detected)
    
    cv.waitKey(0) #Espera que uma tecla seja pressionada.
    cv.destroyAllWindows() #Fecha as janelas após pressionar qualquer tecla.

    return 0
if __name__ == "__main__":
    main()

Contours: (array([[[ 74, 882]],

       [[ 73, 883]],

       [[ 72, 883]],

       [[ 71, 884]],

       [[ 70, 884]],

       [[ 69, 885]],

       [[ 68, 885]],

       [[ 67, 886]],

       [[ 56, 886]],

       [[ 55, 885]],

       [[ 54, 885]],

       [[ 53, 884]],

       [[ 50, 884]],

       [[ 49, 885]],

       [[ 48, 885]],

       [[ 47, 886]],

       [[  0, 886]],

       [[  0, 942]],

       [[ 85, 942]],

       [[ 85, 888]],

       [[ 83, 886]],

       [[ 82, 886]],

       [[ 81, 885]],

       [[ 80, 885]],

       [[ 79, 884]],

       [[ 78, 884]],

       [[ 77, 883]],

       [[ 76, 883]],

       [[ 75, 882]]], dtype=int32), array([[[979, 868]],

       [[978, 869]],

       [[977, 869]],

       [[976, 870]],

       [[975, 870]],

       [[973, 872]],

       [[973, 878]],

       [[975, 880]],

       [[976, 880]],

       [[977, 881]],

       [[978, 881]],

       [[979, 882]],

       [[980, 882]],

       [[981, 881]],

       [[982, 881]],

       